<div style="text-align: center;">
    <h1>Base SIRENE</h1>
</div>


La **base Sirene** est le fournisseur des données d’identité des entreprises et des établissements. 
Elle fait partie des données de référence du Service public de la donnée mis en place par la loi pour une République numérique.
Voir plus d'informations sur le site de **[l'Insee](https://www.insee.fr/fr/information/3591226#:~:text=Elle%20donne%20acc%C3%A8s%20aux%20donn%C3%A9es,r%C3%A9pertoire%20interadministratif%20Sirene%20depuis%201973.)**.

Le site [sirene.fr](https://www.sirene.fr/sirene/public/accueil) permet de créer en ligne des fichiers comprenant au maximum 200 000 établissements, ce qui n'est pas assez pour nos besoins ici, il faut donc tout télécharger sur [data gouv](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/).

Le but de ce notebook est de télécharger cette base de données et de la nettoyer pour obtenir les informations suivantes :

- **siret** : code d'identification d'un établissement. On distingue ici les entreprises (identifiées par leur numéro SIREN) des établissements (SIRET = SIREN + NIC)
- **codeApe** : activité principale de l'établissement renseignée à l'INPI
- **codeInseeCommune** : code de la commune pour pouvoir localiser les entreprises
- **nomCommercial** : nom de l'entreprise 
- **diffusionCommerciale** : oui / non 
- **adresse** : l'adresse de l'établissement

Au total, on a téléchargé ces informations pour **15 millions d'établissements**.


In [2]:
import pandas as pd
import numpy as np

# 1. Téléchargement des données

On télécharge les données via ce [lien permanent](https://www.data.gouv.fr/fr/datasets/r/0651fb76-bcf3-4f6a-a38d-bc04fa708576) aux serveurs de DataGouv.

La documentation relative au jeu de données est disponible [ici](https://static.data.gouv.fr/resources/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/20230523-095315/description-fichier-stocketablissement.pdf).

In [2]:
# paramètres
url = "https://www.data.gouv.fr/fr/datasets/r/0651fb76-bcf3-4f6a-a38d-bc04fa708576"

usecols_dict = {
    "siret": str,
    "numeroVoieEtablissement": str,
    "typeVoieEtablissement": str,
    "libelleVoieEtablissement": str,
    "codePostalEtablissement": str,
    "libelleCommuneEtablissement": str,
    "codeCommuneEtablissement": str,
    "activitePrincipaleEtablissement": str,
    "denominationUsuelleEtablissement": str,
    "etatAdministratifEtablissement": str,
    "enseigne1Etablissement": str,
    "enseigne2Etablissement": str,
    "enseigne3Etablissement": str,
    "statutDiffusionEtablissement": str,
    "nomenclatureActivitePrincipaleEtablissement": str,
}


df = pd.read_csv(
    url, compression="zip", usecols=list(usecols_dict.keys()), dtype=usecols_dict
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37768657 entries, 0 to 37768656
Data columns (total 15 columns):
 #   Column                                       Dtype 
---  ------                                       ----- 
 0   siret                                        object
 1   statutDiffusionEtablissement                 object
 2   numeroVoieEtablissement                      object
 3   typeVoieEtablissement                        object
 4   libelleVoieEtablissement                     object
 5   codePostalEtablissement                      object
 6   libelleCommuneEtablissement                  object
 7   codeCommuneEtablissement                     object
 8   etatAdministratifEtablissement               object
 9   enseigne1Etablissement                       object
 10  enseigne2Etablissement                       object
 11  enseigne3Etablissement                       object
 12  denominationUsuelleEtablissement             object
 13  activitePrincipaleEtablis

# 2. Définition des variables que l'on souhaite garder et des transformations à appliquer

On garde :

- les établissements actifs
- les établissements catégorisés avec la NAF rev2 (l'ancienne naf concerne très peu d'établissements)

In [3]:
rows = df.shape[0]

nb = df[df.etatAdministratifEtablissement != "A"].shape[0]
print("Part d'établissements fermés : {:.2%}".format(nb / rows))

nb = df[df.nomenclatureActivitePrincipaleEtablissement != "NAFRev2"].shape[0]
print("Part d'établissements 'mal' nomenclaturés : {:.2%}".format(nb / rows))

nb = df[
    (df.nomenclatureActivitePrincipaleEtablissement != "NAFRev2")
    | (df.etatAdministratifEtablissement != "A")
].shape[0]
print("Suppression de {:.2%} des données".format(nb / rows))

# on garde les établissements actifs et NAFRev2
df = df[
    (df.nomenclatureActivitePrincipaleEtablissement == "NAFRev2")
    & (df.etatAdministratifEtablissement == "A")
].drop(
    ["nomenclatureActivitePrincipaleEtablissement", "etatAdministratifEtablissement"],
    axis=1,
)

Part d'établissements fermés : 59.12%
Part d'établissements 'mal' nomenclaturés : 25.46%
Suppression de 59.12% des données


Transformations:

- créer une colonne adresse qui concatène les informations
- créer une colonne nomCommercial qui concatène les informations
- changer les dtypes 
- changer les noms de colonnes


In [ ]:
# 1. Adresses
cols_adresse = [
    "numeroVoieEtablissement",
    "typeVoieEtablissement",
    "libelleVoieEtablissement",
    "codePostalEtablissement",
    "libelleCommuneEtablissement",
]

df["adresse"] = df[cols_adresse].apply(
    lambda x: " ".join(x.dropna().astype(str).replace("[ND]", "")).strip(), axis=1
)

df = df.drop(cols_adresse, axis=1)

# 2. Nom commercial
cols_nom = [
    "enseigne1Etablissement",
    "enseigne2Etablissement",
    "enseigne3Etablissement",
    "denominationUsuelleEtablissement",
]

df["nomCommercial"] = df[cols_nom].apply(
    lambda x: " ".join(x.dropna().astype(str).replace("[ND]", "")).strip(), axis=1
)

df = df.drop(cols_nom, axis=1)

In [25]:
# 3. Changement dtypes
df["statutDiffusionEtablissement"] = (
    df["statutDiffusionEtablissement"].replace(["O", "P"], [1, 0]).astype(bool)
)
df["siret"] = df["siret"].astype(np.int64)

# 4. Nom colonnes
colonnes = {
    "siret": "siret",
    "statutDiffusionEtablissement": "diffusionCommerciale",
    "codeCommuneEtablissement": "codeInseeCommune",
    "activitePrincipaleEtablissement": "codeApe",
    "adresse": "adresse",
    "nomCommercial": "nomCommercial",
}

df.columns = list(colonnes.values())

# 3. Sauvegarde du fichier

In [3]:
df = pd.read_parquet("../data/Données nationales/Sirene.parquet")

In [24]:
df["temp"] = df.adresse.apply(lambda x: True if "[ND]" in str(x) else False)

In [25]:
df[df.temp == True]

,siret,diffusionCommerciale,codeInseeCommune,codeApe,adresse,nomCommercial,temp


In [23]:
df["adresse"] = df["adresse"].str.replace("[ND]", "", n=4)
df["nomCommercial"] = df["nomCommercial"].str.replace("[ND]", "", n=4)

In [26]:
df.drop("temp", axis=1, inplace=True)

In [27]:
df.describe(include="all")

,siret,diffusionCommerciale,codeInseeCommune,codeApe,adresse,nomCommercial
count,1.543815e+07,15438154,15241019,15438154,15438154,15438154
unique,NaN,2,35096,731,6678448,2146419
top,NaN,True,75108,68.20B,,
freq,NaN,14061925,144467,1988515,25382,12667617
mean,6.779862e+13,NaN,NaN,NaN,NaN,NaN
std,2.286939e+13,NaN,NaN,NaN,NaN,NaN
min,3.251750e+10,NaN,NaN,NaN,NaN,NaN
25%,4.518347e+13,NaN,NaN,NaN,NaN,NaN
50%,7.998818e+13,NaN,NaN,NaN,NaN,NaN
75%,8.875430e+13,NaN,NaN,NaN,NaN,NaN


In [28]:
df.to_parquet("../data/Données nationales/Sirene.parquet", index=False)